In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -q kagglehub
!pip install -q git+https://github.com/google/qwix
!pip install -q grain
!pip install -q tensorflow tensorflow-datasets
!pip install -q gcsfs

In [3]:
from kubernetes import client, config, watch
from kubernetes.config import config_exception, kube_config
from datasets import load_dataset
from r2egym.agenthub.runtime.docker import DockerRuntime
from r2egym.agenthub.utils.log import get_logger
from dataclasses import asdict, dataclass, field
from typing import Any
import json
import re
import r2egym
from r2egym.agenthub.environment.env import RepoEnv, EnvArgs
from pprint import pprint
from r2egym.agenthub.action import Action
import os
from transformers import AutoTokenizer
from tunix.rl.rollout.vanilla_rollout import VanillaRollout
import pathwaysutils
import jax
from pprint import pprint
import kagglehub
from tunix.models.qwen3 import model as model_lib
from tunix.models.qwen3 import params as params_lib
import optax
from orbax import checkpoint as ocp
from qwix import lora
from flax import nnx
import tempfile
import jax.numpy as jnp
from tunix.experimental.deep_swe.scripts.gemma2_parser import Gemma2ChatParser
from tunix.generate import sampler as sampler_lib
from tunix.models.gemma import data as data_lib
from tunix.experimental.deep_swe.scripts.swe_agent import SWEAgent
from tunix.experimental.deep_swe.scripts.swe_env import SWEEnv
from tunix.experimental.deep_swe.scripts.chat_template import ChatTemplateParser
import libtpu.sdk as sdk 


2025-07-31 05:05:01,469 - root - INFO - Could not import `wandb`. Logging to W&B not possible.


In [4]:
# pathwaysutils.initialize()
pprint(jax.devices())

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=4, process_index=0, coords=(0,2,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(1,2,0), core_on_chip=0),
 TpuDevice(id=6, process_index=0, coords=(0,3,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,3,0), core_on_chip=0)]


In [5]:
sdk.monitoring.list_supported_metrics()

['tensorcore_util',
 'duty_cycle_pct',
 'buffer_transfer_latency',
 'hbm_capacity_total',
 'hbm_capacity_usage',
 'hlo_execution_timing',
 'hlo_queue_size']

In [ ]:
def print_hbm_usage():
    metric_hbm = sdk.monitoring.get_metric('hbm_capacity_total')
    metric_hbm_usage = sdk.monitoring.get_metric('hbm_capacity_usage')
    used = int(metric_hbm_usage.data()[0])
    total = int(metric_hbm.data()[0])
    print(f"HBM Usage: {used / total * 100:.2f}%, Total: {total / 1024**3:.2f} GB, Used: {used / 1024**3:.2f} GB")

In [24]:
print_hbm_usage()

HBM Usage: 15.65%, Total: 31.25 GB, Used: 4.89 GB


# Before start
- Making sure that pw headless cluster port forwarding is set up, if not, check: tunix/experimental/deep_swe/docs/local_development_guide.md
- Make sure kubeconfig environment is set up

In [ ]:
assert os.getenv('KUBECONFIG') is not None
assert os.getenv('JAX_BACKEND_TARGET') is not None

# assert os.getenv('JAX_PLATFORMS') is not None



# How to start k8s cluster and get kube config
https://docs.google.com/document/d/1c6NOGa6N0HeFPFIcYXTMW9ldrndiYB38ciMMa5hE3TQ/edit?tab=t.0
- create_cpu_cluster()
- get_cpu_kubeconfig()

**NOTE: with xpk, by default it returns ipv4 addresses in kubeconfig, wheras borg job running on ipv6 only environment. So I have to use VM jupyter notebook, as VM also runs on ipv4. Need to eiter start xpk with ipv6 or allow ipv4 for the borg job. They need to be consistent.**

In [11]:
DATASET_CACHE = os.getenv('DATASET_CACHE', '/tmp/dataset_cache')
print(DATASET_CACHE)
TASKS_TO_PROCESS = 100

/mnt/disks/extra-disk/data


In [10]:
config.load_kube_config()
k8s_client = client.CoreV1Api()

# Load dataset

In [12]:
dataset = load_dataset("R2E-Gym/R2E-Gym-V1", split="train", cache_dir=DATASET_CACHE, num_proc=32)
entries = []
unique_images = set()
for i, entry in enumerate(dataset):
  if "docker_image" in entry:
    unique_images.add(entry["docker_image"])
    entries.append(entry)
  if i >= TASKS_TO_PROCESS - 1:
    break
unique_images = list(unique_images)
print(f"Found {len(unique_images)} unique Docker images to download")
IDS = [f"task-{i}" for i in range(len(entries))]

Found 100 unique Docker images to download


# Load model

In [ ]:
# Log in
if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
  kagglehub.login()

In [13]:
kaggle_ckpt_path = kagglehub.model_download("qwen-lm/qwen-3/transformers/14b")
print(kaggle_ckpt_path)

/mnt/disks/extra-disk/cache/models/qwen-lm/qwen-3/transformers/14b/1


In [14]:

# This is a workaround. The checkpoints on Kaggle don't work with NNX. So, we
# load the model, save the checkpoint locally, and then reload the model
# (sharded).

mesh = jax.make_mesh((2, len(jax.devices())//2), ('fsdp', 'tp'))
model_config = model_lib.ModelConfig.qwen3_14_b()
qwen3 = params_lib.create_model_from_safe_tensors(kaggle_ckpt_path, model_config, mesh=mesh)
tokenizer = AutoTokenizer.from_pretrained(kaggle_ckpt_path)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 443/443 [00:07<00:00, 61.32it/s]


Sharding: {'embedder': {'input_embedding': NamedSharding(mesh=Mesh('fsdp': 2, 'tp': 4, axis_types=(Auto, Auto)), spec=PartitionSpec('tp', 'fsdp'), memory_kind=device)}, 'final_norm': {'w': NamedSharding(mesh=Mesh('fsdp': 2, 'tp': 4, axis_types=(Auto, Auto)), spec=PartitionSpec('tp',), memory_kind=device)}, 'layers': {0: {'attn': {'k_norm': {'w': NamedSharding(mesh=Mesh('fsdp': 2, 'tp': 4, axis_types=(Auto, Auto)), spec=PartitionSpec('tp',), memory_kind=device)}, 'k_proj': {'w': NamedSharding(mesh=Mesh('fsdp': 2, 'tp': 4, axis_types=(Auto, Auto)), spec=PartitionSpec('tp', 'fsdp', None), memory_kind=device)}, 'o_proj': {'w': NamedSharding(mesh=Mesh('fsdp': 2, 'tp': 4, axis_types=(Auto, Auto)), spec=PartitionSpec('tp', None, 'fsdp'), memory_kind=device)}, 'q_norm': {'w': NamedSharding(mesh=Mesh('fsdp': 2, 'tp': 4, axis_types=(Auto, Auto)), spec=PartitionSpec('tp',), memory_kind=device)}, 'q_proj': {'w': NamedSharding(mesh=Mesh('fsdp': 2, 'tp': 4, axis_types=(Auto, Auto)), spec=PartitionSp

In [19]:
print_hbm_usage()

HBM Usage: 0.00%, Total: 31.25 GB, Used: 0.00 GB


In [ ]:
MAX_PROMPT_LENGTH = 10000
MAX_RESPONSE_LENGTH = 10000
# gemma_tokenizer = data_lib.GemmaTokenizer()
sampler = sampler_lib.Sampler(
    transformer=qwen3,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=MAX_PROMPT_LENGTH + MAX_RESPONSE_LENGTH + 256, # got from /home/annyan_google_com/development/rllm/rllm/engine/agent_execution_engine.py
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

def templatize(prompts):
  out = []
  for p in prompts:
    out.append(
        tokenizer.apply_chat_template(
          [
              {"role": "user", "content": p},
          ],
          tokenize=False,
          add_generation_prompt=True,
          enable_thinking=True,
      )
    )
  return out

def generate_model_response(input) -> str:
    out_data = sampler(
        templatize([input]),
        max_prompt_length=MAX_PROMPT_LENGTH,
        total_generation_steps=MAX_RESPONSE_LENGTH,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        echo=False,
        seed=jax.random.PRNGKey(0)
    )
    output = out_data.text
    return output[0]

In [ ]:
generate_model_response("which is bigger, the sun or the moon? return me some real statistics.")

In [ ]:
# to start a k8s pod with an entry
# because I want to use cached pod, so I will skip executing this cell most of the time, unless I want to start a new pod
runtime = DockerRuntime(ds=entries[0], command=["/bin/bash", "-l"], logger=get_logger(), backend="kubernetes", id=IDS[0])

In [ ]:
# query one pod name, which is previously created
pod = k8s_client.read_namespaced_pod(namespace="default", name=IDS[0], _request_timeout=60)
print(pod.status.container_statuses[0])

In [ ]:

from tunix.experimental.deep_swe.scripts.swe_agent import SWEAgent
from tunix.experimental.deep_swe.scripts.swe_env import SWEEnv

In [ ]:
agent = SWEAgent(
)
env = SWEEnv(entry=entries[0])

In [ ]:
observation, info = env.reset()

In [ ]:
# pprint(f"observation: {observation}")
# pprint(f"info: {info}")
agent.reset()

In [ ]:
parser = ChatTemplateParser.get_parser(tokenizer, disable_thinking=False)

In [ ]:
agent.update_from_env(
            observation=observation,  # Raw observation from environment
            reward=0.0,
            done=False,
            info=info,
        )
prompt_messages = agent.chat_completions.copy()
prompt_str = parser.parse(prompt_messages, add_generation_prompt=True, is_first_msg=True)
response = generate_model_response(prompt_str)



In [ ]:
response

In [ ]:
pattern = re.compile(r"(?s)(<function=.*?</function>)")
match = pattern.search(response)

In [ ]:
match.group(1)

In [ ]:
action = agent.update_from_model(response)
pprint(f"action before: {action}")
action = action.action
pprint(f"action after: {action}")

In [ ]:
next_observation, reward, done, info = env.step(action)

In [ ]:
pprint(next_observation)
pprint(reward)
pprint(done)
pprint(info)

In [33]:
from tunix.experimental.deep_swe.rllm.environments.swe.swe import SWEEnv
from tunix.experimental.deep_swe.rllm.agents.swe_agent import SWEAgent
from tunix.experimental.deep_swe.rllm.engine.agent_execution_engine import AgentExecutionEngine
from concurrent.futures import ThreadPoolExecutor, as_completed

In [29]:
dataset = entries[:5] # change to any number of entries you want to process


In [31]:
def _create_env(idx):
    entry = dataset[idx]
    return idx, SWEEnv(
        entry = entry,
        idx = idx
    )
def _create_agent(idx):
    return idx, SWEAgent()
agents = [None] * len(dataset)
envs = [None] * len(dataset)
with ThreadPoolExecutor(max_workers=8) as executor:
    env_futures = [executor.submit(_create_env, i) for i in range(len(dataset))]
    for future in as_completed(env_futures):
        idx, env = future.result()
        envs[idx] = env
with ThreadPoolExecutor(max_workers=64) as executor:
    agent_futures = [executor.submit(_create_agent, i) for i in range(len(envs))]
    for future in as_completed(agent_futures):
        idx, agent = future.result()
        agents[idx] = agent

In [34]:
agent_execution_engine = AgentExecutionEngine(
    qwen3=qwen3,
    model_config=model_config,
    tokenizer=tokenizer,
)

model_name: /mnt/disks/extra-disk/cache/models/qwen-lm/qwen-3/transformers/14b/1, tokenizer_cls: qwen2tokenizerfast
Using QwenChatTemplateParser for /mnt/disks/extra-disk/cache/models/qwen-lm/qwen-3/transformers/14b/1


In [35]:
agent_execution_engine.update_envs_and_agents(envs, agents)

Total tasks to run: 5


In [ ]:
trajectories = []
async for trajectory in agent_execution_engine.trajectory_generator():
    print(trajectory)
    trajectories.append(trajectory)

Running 5 trajectories with max concurrency of 5...
Using container name: b9035425-a50f-49de-87bb-5ab659a0ef4f for Kubernetes pod
Using container name: a9c8a558-43ac-4da0-bd06-2d49e145e23f for Kubernetes pod
Using container name: 3a8befaa-d69d-4b24-99ba-fd99951ac51d for Kubernetes pod
Using container name: a9888942-618c-4b58-932c-505abf0585ba for Kubernetes pod
Using container name: 71d5d244-b9ad-4974-a90d-2365f3878932 for Kubernetes pod
